In [ ]:
%pip install torch
%pip install fairscale
%pip install fire
%pip install sentencepiece

In [ ]:
import subprocess
import json
import os
from dotenv import load_dotenv

load_dotenv()


LLAMA_RUNNER = os.environ['LLAMA_RUNNER']
LLAMA_DATA = os.environ['LLAMA_DATA']


'''
LLAMA PIPELINE
'''

def llama(input):

    with open(LLAMA_DATA, 'w') as file:
        json.dump(input, file, indent=4)

    arguments = ['torchrun', 
                '--nproc_per_node=1', 
                f'{LLAMA_RUNNER}', 
                '--max_seq_len=512', 
                '--max_batch_size=6']
    result = subprocess.run(arguments, capture_output=True, text=True)

    print("Error:", result.stderr)
    print("Return Code:", result.returncode)

    return result.stdout


def parse_llama_result(data:str, remove_first_parapgraph=False):
    data = data[data.find('[@DATABEGIN]'):data.find('[@DATAEND]')]
    story = data[data.find('[@RESPONSEBEGIN]'):data.find('[@RESPONSEEND]')]
    if remove_first_parapgraph:
        story = story[story.find('[SCENE'):]
    return story


In [23]:
prompt = 'animals overrunning a village and destroying its crops'

example = """{
    "characters": [
        {"name": "Radke"}
    ],
    "script": [ 
        {"caption": "A sheep looking at the wheels of cheese in a supermarket.", 
         "dialogue": [
            {"character": "Radke", "text": "In the mist-enshrouded hills of an ancient land, there lies a mystery as old as time itself. Behold the enigmatic sheep, creatures shrouded in the lore and legend of yesteryears."}
        ]
        },
        {"caption": "A sheep wearing a wizard hat casting magic spells with a ritual circle beneath them in Stonehenge", 
         "dialogue": [
            {"character": "Radke", "text": "For centuries, scholars and mystics have pondered over their serene demeanors and unassuming presence."},
            {"character": "Radke", "text": "What arcane knowledge do they guard beneath their fluffy exteriors?"}
        ]
        },
        {"caption": "A man looking at sheep eating grass around a field.", 
         "dialogue": [
            {"character": "Radke", "text": "Join us, as we unravel the ancient mystery of the sheep."}
        ]
        }
    ]
}"""

data = [
    [
        {"role": "user", "content": f"write a story about {prompt}, breaking it apart into separate paragraphs such that each scene is its own paragraph. Begin each paragraph of the story with the label [SCENE]"}
    ],
]


data = [
    [
        {"role": "user", "content": f"write a story about {prompt}, and present it in the following format: {example}"}
    ],
]

story = llama(data)
story_parsed = parse_llama_result(story)


data = [
    [
        {"role": "user", "content": f"generate the captions for a series of images that would pair well with the following story such that there is one caption per paragraph, Begin each caption with the label [SCENE]: \n{story_parsed}."}
    ],
]

captions = llama(data)
captions_parsed = parse_llama_result(captions)


Error: /home/eggyrepublic/ERU/RPI/Computational-Creativity/.conda/lib/python3.10/site-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)

Return Code: 0
Error: /home/eggyrepublic/ERU/RPI/Computational-Creativity/.conda/lib/python3.10/site-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)

Return Code: 0
Error: /home/eggyrepublic/ERU/RPI/Computational-Creativity/.conda/lib/python3.10/site-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, plea

In [24]:
print(f"story parsed: \n{story_parsed}\nEND")
print(f"captions parsed: \n{captions_parsed}\nEND")

story parsed: 
[SCENE 1]
The village of Greenhaven was a peaceful and prosperous place, nestled in the heart of a lush green valley. The villagers worked hard to cultivate their fields and tend to their livestock, but they had never faced a threat like the one that was coming their way. As the sun began to set on a warm summer evening, a group of wild animals emerged from the nearby forest, their eyes fixed on the village.

[SCENE 2]
The animals, led by a fierce and cunning wolf, moved quickly and silently towards the village. They had been drawn by the scent of fresh bread and the sound of laughter, and they were determined to claim Greenhaven as their own. The villagers, caught off guard, watched in horror as the animals surrounded their homes and began to destroy everything in sight. Crops were trampled, livestock were killed, and homes were reduced to rubble.

[SCENE 3]
The villagers tried to fight back, but they were no match for the ferocious animals. The wolf led the charge, its